In [1]:
#### stat the proportion of each tissue
total_asm6a = "/Charles/project/ASm6A/ASm6A/common_ASm6A/totaltissues/common_ASm6A.bed"
asm6a_dir = "/Charles/project/ASm6A/ASm6A/"
result_dir = "/Charles/project/ASm6A/ASm6A/common_ASm6A/totaltissues/by_sample/"

asm6a_list = glob.glob("%s/*.bed" % asm6a_dir)
for bed in asm6a_list:
    sample = os.path.basename(bed).split(".bed")[0]
    count=int(os.popen("bedtools intersect -a %s -b %s -wa|sort -u|wc -l"%(bed,total_asm6a)).read().split()[0])
    print(sample, count)
    res = os.path.join(result_dir, os.path.basename(bed))
    os.system("bedtools intersect -a %s -b %s -wa -wb > %s" % (bed, total_asm6a, res))

Kidney_2 13
Liver_3 51
Spleen_3 220
stomach_IP_4 279
Appendix-3-2 38
Liver_1 39
Cerebellum-5-3 146
Adrenal_gland-1-1 286
FrontalCortex_3 134
liver_IP_3 243
kidney_IP_4 392
Cerebellum_3 182
Aorta-4-2 74
Muscle_1 22
Tongue-2-1 33
Cerebrum-6-3 45
Lung_2 139
Hypothalamus-5-3 70
Heart-4-2 58
Cerebrum-5-3 91
Spleen-3-2 17
Cerebellum_1 99
Heart-1-1 263
Urinary_bladder-2-1 18
Lung-4-4 6
Rectum-4-2 18
Kidney_3 128
Heart_2 105
Colon-3-2 48
heart_IP_2 289
Spleen_1 123
lung_IP_5 134
Spleen-2-1 41
kidney_IP_2 488
Skin-1-1 301
Kidney_1 69
Brainstem-5-3 110
Stomach-5-3 45
Muscle_3 94
placenta_IP_6 240
Adipose-1-1 247
FrontalCortex_2 228
Lung-2-1 15
Muscle-3-2 14
brain_IP_1 211
placenta_IP_2 416
Spleen_2 1
Esophagus-4-2 34
muscle_IP_5 391
heart_IP_3 242
Heart_1 80
Lung-2-4 37
heart_IP_1 108
Spleen-1-1 183
Liver_2 5
Trachea-5-3 67
Aorta-1-1 128
liver_IP_2 240
Jejunum-5-3 19
Esophagus-3-2 41
Cerebellum_2 127
Jejunum-4-2 24
Urinary_bladder-5-3 22
Duodenum-5-3 10
Thyroid_gland-5-3 57
Thyroid_gland-4-2 45


In [10]:
#### 
# select ASm6As that have the same trend across samples which contains the same ASm6A
import math
from scipy import stats

#### ASm6A
asm6a_dir = "/Charles/project/ASm6A/ASm6A/"
os.chdir(asm6a_dir)
result_dir = "/Charles/project/ASm6A/ASm6A/common_ASm6A/totaltissues/compare_imbalance/"
# common_asm6a = "/Charles/project/ASm6A/ASm6A/common_ASm6A/totaltissues/common_ASm6A.bed"#############
common_asm6a = "/Charles/project/ASm6A/ASm6A/common_ASm6A/totaltissues/common_ASm6A_70%.bed"
#############################################################################################
def read_commonASm6A():
    df = pd.read_table(common_asm6a, header=None)
    df['term'] = df.iloc[:,0] + ":" + df.iloc[:,2].astype(str)
    df['mark'] = df.iloc[:,3]
    return dict(zip(df['term'].tolist(), df['mark'].tolist()))
    
def calculate_score(df):
    df['score'] = 0.5 - df.iloc[:,3].str.split(";").str[1].astype(float)
    return df[['term','score']]

def gener_score_dict(df):
    res_dict, final_dict = {}, {}
    for i, values in df.iterrows():
        res_dict[values['term']] = res_dict.get(values['term'], []) + [values['score']]
    ### average score of each SNP
    for term, score_list in res_dict.items():
        final_dict[term] = abs(sum(score_list)/len(score_list))
    return final_dict

commASm6A_dict = read_commonASm6A()
comm_list = list(commASm6A_dict.keys())
bed_list = glob.glob("%s/*.bed" % asm6a_dir)
case_list, cont_list = [], []
for bed in bed_list:
    sample = os.path.basename(bed)
    df = pd.read_table(bed, header=None)
    df['term'] = df.iloc[:,0] + ":" + df.iloc[:,2].astype(str) # + ":" + df.iloc[:,3].str.split(";").str[0]
    df_case, df_cont = df[df['term'].isin(comm_list)], df[~df['term'].isin(comm_list)]
    df_case, df_cont = calculate_score(df_case), calculate_score(df_cont)
    case_list.append(df_case)
    cont_list.append(df_cont)
df_case, df_cont = pd.concat(case_list), pd.concat(cont_list)
case_dict, cont_dict = gener_score_dict(df_case), gener_score_dict(df_cont)
case_values, cont_values = list(case_dict.values()), list(cont_dict.values())
print(stats.mannwhitneyu(case_values, cont_values))
print(np.mean(case_values), np.mean(cont_values))
### write to file
file_list = [os.path.join(result_dir, "case_scores_70%.txt"), os.path.join(result_dir, "cont_scores_70%.txt")]
score_list = [case_values, cont_values]
for i in range(len(file_list)):
    with open(file_list[i], 'w') as fw:
        fw.writelines([str(x) + "\n" for x in score_list[i]])

/home/galaxy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


MannwhitneyuResult(statistic=11532031.5, pvalue=1.1654550005193945e-86)
0.15028444009501818 0.12659723627668837
